<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/Decision_Trees_LGB_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [3]:
os.listdir("drive/My Drive/colab_datas/IE 492")

['Comparison of methods.pdf',
 '1-s2.0-S1364032114000914-main.pdf',
 '1-s2.0-S0360544209002539-main.pdf',
 '1-s2.0-S1876610211043116-main.pdf',
 '1-s2.0-S0305048311001423-main.pdf',
 'weather_turkey_processed.zip',
 'Meta_L_makaleler',
 'all_data.feather',
 'UECM-15012015-20022020.csv',
 'RealTimeConsumption-01122014-09032020.csv',
 'updated_support-date_based_information.r',
 'ozel_gunler',
 'RealTimeConsumption_unholidayized.csv',
 'RealTimeConsumption_unhol_decomposed.csv']

In [4]:
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_unholidayized.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/updated_support-date_based_information.r","./")

'./updated_support-date_based_information.r'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

# Evaluation function

In [0]:
!pip install graphviz

from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
from sklearn.utils import check_arrays
from sklearn import tree

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = check_arrays(y_true, y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

eval_df=[]
def eval_function(name_,y_true, y_pred,valid_df):
    """
    docstring denir buraya func paramlarının neler oldugunu yazarsınız
    name_:kullanılan modelin ismi
    y_true:actual values
    y_pred:predicted values
    valid_df:modele verdiginiz sekliyle validation dataframe'i ve yanına datetime'ı
    """
    rmse_=np.sqrt(mean_squared_error(y_true,y_pred))
    rmsle_=np.sqrt(mean_squared_log_error(y_true,y_pred))
    mae_=mean_absolute_error(y_true,y_pred)
    mape_=mean_absolute_percentage_error(y_true, y_pred)

    print("RMSE: ",rmse_)
    print("RMSLE: ",rmsle_)
    print("MAE: ",mae_)
    print("MAPE: ",mape_)
    sns.residplot(y_true,y_pred)
    plt.pause(0.01)
    fig,ax=plt.subplots(nrows=6,figsize=(16,10))
    valid_copy=valid_df.reset_index(drop=True).copy()
    for m1,m2 in np.arange(1,13).reshape((6,2)):
        datetime_=valid_copy.loc[valid_copy["Month"].between(m1,m2)]["Datetime"]
        sns.lineplot(x=y_true[datetime_.index],y=datetime_,color="green",ax=ax[int(m2/2)-1])
        sns.lineplot(x=y_pred[datetime_.index],y=datetime_,color="red",ax=ax[int(m2/2)-1])
    plt.pause(0.01)
    res_model=DecisionTreeRegressor(max_depth=4)

    res_model.fit(valid_df.drop(columns=["Datetime"]),y_true-y_pred)
    tree.plot_model(tree)
    plt.pause(0.01)
    return name_, rmse_,rmsle_,mae_,mape_

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12]])

# Consumption df

In [0]:
# bruuuuuuuh
consumption_df=pd.read_csv("RealTimeConsumption_unholidayized.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]+consumption_df.columns.tolist()[3:]
consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"],format="%d.%m.%Y %H:%M")
consumption_df

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29745.882083,2015-12-31 00:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
1,31.12.2015,01:00,27940.432083,2015-12-31 01:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
2,31.12.2015,02:00,26671.532083,2015-12-31 02:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
3,31.12.2015,03:00,26246.622083,2015-12-31 03:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
4,31.12.2015,04:00,26026.542083,2015-12-31 04:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.00000,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19:00,37743.533050,2020-03-03 19:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36572,03.03.2020,20:00,37360.113050,2020-03-03 20:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36573,03.03.2020,21:00,36388.473050,2020-03-03 21:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
36574,03.03.2020,22:00,35241.103050,2020-03-03 22:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.00000,0.0,0.0,-0.0,193.25695,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.000000,0.0
